In [1]:
import os,sys,time,codecs,random
from queue import Queue
from collections import namedtuple, Counter
from itertools import count
import numpy as np
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
os.listdir()

['.ipynb_checkpoints',
 'BFS',
 'DeepPath.ipynb',
 'env.py',
 'evaluate.py',
 'fact_prediction_eval.py',
 'link_prediction_eval.sh',
 'networks.py',
 'pathfinder.sh',
 'policy_agent.py',
 'sl_policy.py',
 'transE_eval.py',
 'transR_eval.py',
 'transX_eval.py',
 'utils.py']

In [3]:
os.listdir('./BFS/')

['BFS.py',
 'BFS.pyc',
 'DFS.pyc',
 'full_data.txt',
 'KB.py',
 'KB.pyc',
 'README',
 'run.py',
 '__init__.py',
 '__init__.pyc']

In [4]:
relation = 'concept_worksfor'

In [5]:
data_path =  '../../NELL-995/'
task_path = data_path + 'tasks/' + relation +'/'

#### pathfinder.sh

#### KB.py

In [6]:
class KB(object):
    def __init__(self):
        self.entities = {} # {'实体id':[Path.{relation,entity2},]}

    def addRelation(self, entity1, relation, entity2):
        # add direct connections
        if entity1 in self.entities.keys():
            self.entities[entity1].append(Path(relation, entity2))
        else:
            # entities{entity1：Path{.relation.connected_entity}}
            self.entities[entity1] = [Path(relation, entity2)] 
           

    def getPathsFrom(self, entity):
        return self.entities[entity]

    def removePath(self, entity1, entity2):
        # remove direct connection between e1 and e2
        for idx, path in enumerate(self.entities[entity1]):
            if(path.connected_entity == entity2):
                del self.entities[entity1][idx]
                break
        for idx, path in enumerate(self.entities[entity2]):
            if(path.connected_entity == entity1):
                del self.entities[entity2][idx]
                break

    def pickRandomIntermediatesBetween(self, entity1, entity2, num):
        # TO DO: COULD BE IMPROVED BY NARROWING THE RANGE OF
        # RANDOM EACH TIME ITERATIVELY CHOOSE AN INTERMEDIATE  
        if num > len(self.entities) - 2:
            raise ValueError('Number of Intermediates picked is larger than possible',\
                             'num_entities: {}'.format(len(self.entities)), \
                             'num_itermediates: {}'.format(num))
        return random.sample(set(self.entities.keys())-set([entity1,entity2]),num) # non-return samples

    def __str__(self):
        return ''.join([entity+','.join([str(path) for path in self.entities[entity]]) \
                        for entity in self.entities])

class Path(object):
    def __init__(self, relation, connected_entity):
        self.relation = relation
        self.connected_entity = connected_entity

    def __str__(self):
        return "rel:{},next_entity:{}".format(self.relation, self.connected_entity)

    __repr__ = __str__

#### BFS.py

In [26]:
class foundPaths(object):
    def __init__(self, kb):
        # {entity:status}
        # status:(isFound,prevNode,relation)
        self.entities = dict([(entity,(False,'','')) \
                              for entity in kb.entities.keys()])

    def isFound(self, entity):
        return self.entities[entity][0]

    def markFound(self, entity, prevNode, relation):
        self.entities[entity] = (True, prevNode, relation)

    def reconstructPath(self, entity1, entity2): # after BFS
        curNode,entity_list,path_list = entity2,[entity2],[] # from tail
        while(curNode != entity1):       # status:(isFound,prevNode, relation)
            path_list.append(self.entities[curNode][2]) # relation
            curNode = self.entities[curNode][1]         # prevNode
            entity_list.append(curNode)
        return entity_list[::-1],path_list[::-1]

    def __str__(self):
        return ''.join([entity + "[{},{},{}]".format(status[0],status[1],status[2]) \
                        for entity, status in self.entities.iteritems()])

def BFS(kb, entity1, entity2):
    '''
    input: kb=KB(),head,tail
    output: (True, entity_list, path_list)
    '''
    path_finder = foundPaths(kb);path_finder.markFound(entity1, None, None)
    q = Queue();q.put(entity1)
    while(not q.empty()):
        curNode = q.get()
        for path in kb.getPathsFrom(curNode): # get connections
            connectRelation,nextEntity = path.relation,path.connected_entity
            if(not path_finder.isFound(nextEntity)): # put for continue search
                q.put(nextEntity)
                path_finder.markFound(nextEntity, curNode, connectRelation)
            if(nextEntity == entity2): # arrive tail
                entity_list, path_list = path_finder.reconstructPath(entity1, entity2)
                return (True, entity_list, path_list)
    return (False, None, None)

#### env.py

In [8]:
class Env(object):
    """knowledge graph environment definition"""
    def __init__(self, data_path, relation='concept:worksfor'):
        self.entity2id_ = entity2id = dict([(line.split()[0],int(line.split()[1])) for line in \
                  codecs.open(data_path+'entity2id.txt','r',encoding='utf-8') if len(line.split()) == 2])
        self.relation2id_ = relation2id = dict([(line.split()[0],int(line.split()[1])) for line in \
                    codecs.open(data_path+'relation2id.txt','r',encoding='utf-8') if len(line.split()) == 2])
        self.relations = list(self.relation2id_.keys())
        self.entity2vec = np.loadtxt(data_path + 'entity2vec.bern')
        self.relation2vec = np.loadtxt(data_path + 'relation2vec.bern')

        self.path = []
        self.path_relations = []

        # kb_env_rl filter:rel
        self.kb = [line.rsplit() for line in codecs.open(data_path+'kb_env_rl.txt','r',encoding='utf-8')\
        if len(line.split()) == 2 and line.split()[2] != relation and line.split()[2] != relation+'_inv']

        self.die = 0 # record how many times does the agent choose an invalid path

    def interact(self, state, action):
        '''
        This function process the interact from the agent
        state: is [current_position, target_position] 
        action: an integer
        return: (reward, [new_postion, target_position], done)
        '''
        done = 0 # Whether the episode has finished
        curr_pos,target_pos = state[:-1]
        chosed_relation = self.relations[action]
        choices = []
        for triple in self.kb:
            if curr_pos == self.entity2id_[triple[0]] \
                and triple[2] == chosed_relation \
                and triple[1] in self.entity2id_:
                choices.append(self.entity2id_[triple[1]])
        if len(choices) == 0:
            reward = -1
            self.die += 1
            next_state = state # stay in the initial state
            next_state[-1] = self.die
            return (reward, next_state, done)
        else: # find a valid step
            next_pos = random.choice(choices)
            self.path.append(chosed_relation + ' -> ' + next_pos)
            self.path_relations.append(chosed_relation)
            print('Find a valid step:',path,'Action index:',action)
            self.die = 0
            reward = 0
            next_state = [next_pos, target_pos, self.die]

            if next_pos == target_pos:
                print('Find a path:',self.path)
                done = 1
                reward = 0
                next_state = None
            return (reward, next_state, done)

    def idx_state(self, idx_list):
        if idx_list != None:
            curr = self.entity2vec[idx_list[0],:]
            targ = self.entity2vec[idx_list[1],:]
            return np.expand_dims(np.concatenate((curr, targ - curr)),axis=0)
        else:
            return None

    def get_valid_actions(self, entityID): # valid action space <= action space
        actions = set()
        for triple in self.kb:
            e1_idx = self.entity2id_[triple[0]]
            if e1_idx == entityID:
                actions.add(self.relation2id_[triple[2]])
        return np.array(list(actions))

    def path_embedding(self, path):
        embeddings = [self.relation2vec[self.relation2id_[relation],:] for relation in path]
        embeddings = np.reshape(embeddings, (-1,embedding_dim))
        path_encoding = np.sum(embeddings, axis=0)
        return np.reshape(path_encoding,(-1, embedding_dim))

In [44]:
env = Env(data_path, relation.replace('_',':'))

#### utils.py

In [39]:
# hyperparameters
state_dim = 200
action_space = 400
eps_start = 1
eps_end = 0.1
epe_decay = 1000
replay_memory_size = 10000
batch_size = 128
embedding_dim = 100
gamma = 0.99
target_update_freq = 1000
max_steps = 50
max_steps_test = 50

Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

def l2_distance(e1, e2):
    return np.sqrt(np.sum(np.square(e1 - e2)))

def compare(v1, v2):
    return sum(v1 == v2)

def prob_norm(probs):
    return probs/sum(probs)

def teacher(e1, e2, num_paths, env, path = None):
    kb = KB()
    [kb.addRelation(line.rsplit()[0],line.rsplit()[1],line.rsplit()[2]) \
     for line in codecs.open(path,'r',encoding='utf-8')]
    # Bi-BFS path collect
    intermediates = kb.pickRandomIntermediatesBetween(e1, e2, num_paths)
    print('intermediates:',intermediates)      
    entity_lists = [];path_lists = []
    for i in range(num_paths):
        suc1, entity_list1, path_list1 = BFS(kb, e1, intermediates[i]);print('{}:BFS left done'.format(i))
        suc2, entity_list2, path_list2 = BFS(kb, intermediates[i], e2);print('{}:BFS right done'.format(i))
        if suc1 and suc2:
            entity_lists.append(entity_list1 + entity_list2[1:])
            path_lists.append(path_list1 + path_list2)
    print('BFS found paths:', len(path_lists))
    # clean the path 
    # duplicate
    # drop [min:max]
    print('path clean')
    entity_lists_new = []
    path_lists_new = []
    for entities, relations in zip(entity_lists, path_lists):
        path = [entities[int(i/2)] if i%2 == 0 else relations[int(i/2)]\
                    for i in range(len(entities)+len(relations))]
        entity_stats = Counter(entities).items()
        duplicate_ents = [item for item in entity_stats if item[1]!=1]
        duplicate_ents.sort(key = lambda x:x[1], reverse=True)
        for item in duplicate_ents:
            ent = item[0]
            ent_idx = [i for i,x in enumerate(path) if x == ent]
            if len(ent_idx)!=0:
                min_idx = min(ent_idx)
                max_idx = max(ent_idx)
                if min_idx!=max_idx:
                    path = path[:min_idx] + path[max_idx:]
        entities_new = []
        relations_new = []
        for idx, item in enumerate(path):
            if idx%2 == 0:
                entities_new.append(item)
            else:
                relations_new.append(item)
        entity_lists_new.append(entities_new);path_lists_new.append(relations_new)
    print('len(entities):',len(entity_lists_new),'len(paths):',len(path_lists_new))
    # episodes
    print('collect episodes')
    good_episodes = []
    targetID = env.entity2id_[e2]
    for path in zip(entity_lists_new,path_lists_new):
        good_episode = []
        for i in range(len(path[0]) -1):
            currID = env.entity2id_[path[0][i]];nextID = env.entity2id_[path[0][i+1]]
            state_curr = [currID, targetID, 0];state_next = [nextID, targetID, 0]
            actionID = env.relation2id_[path[1][i]]
            good_episode.append(Transition(state = env.idx_state(state_curr),\
                                           action = actionID, \
                                           next_state = env.idx_state(state_next), \
                                           reward = 1)) # each time step reward==1
        good_episodes.append(good_episode)
    return good_episodes

def path_clean(path):
    rel_ents = path.split(' -> ')
    relations = []
    entities = []
    for idx, item in enumerate(rel_ents):
        if idx%2 == 0:
            relations.append(item)
        else:
            entities.append(item)
    entity_stats = Counter(entities).items()
    duplicate_ents = [item for item in entity_stats if item[1]!=1]
    duplicate_ents.sort(key = lambda x:x[1], reverse=True)
    for item in duplicate_ents:
        ent = item[0]
        ent_idx = [i for i, x in enumerate(rel_ents) if x == ent]
        if len(ent_idx)!=0:
            min_idx = min(ent_idx)
            max_idx = max(ent_idx)
            if min_idx!=max_idx:
                rel_ents = rel_ents[:min_idx] + rel_ents[max_idx:]
    return ' -> '.join(rel_ents)

#### networks.py

In [10]:
def policy_nn(state, state_dim, action_dim, initializer):
    """
    策略网络(P(a_t|s_t;theta))
    state_dim -relu-> 512 -relu-> 1024 -softmax-> action_dim
    state -> action_prob [action_dim]
    action_dim == 关系数量
    """
    w1 = tf.get_variable('W1', [state_dim, 512], initializer = initializer, regularizer=tf.contrib.layers.l2_regularizer(0.01))
    b1 = tf.get_variable('b1', [512], initializer = tf.constant_initializer(0.0))
    h1 = tf.nn.relu(tf.matmul(state, w1) + b1)
    w2 = tf.get_variable('w2', [512, 1024], initializer = initializer, regularizer=tf.contrib.layers.l2_regularizer(0.01))
    b2 = tf.get_variable('b2', [1024], initializer = tf.constant_initializer(0.0))
    h2 = tf.nn.relu(tf.matmul(h1, w2) + b2)
    w3 = tf.get_variable('w3', [1024, action_dim], initializer = initializer, regularizer=tf.contrib.layers.l2_regularizer(0.01))
    b3 = tf.get_variable('b3', [action_dim], initializer = tf.constant_initializer(0.0))
    action_prob = tf.nn.softmax(tf.matmul(h2,w3) + b3)
    return action_prob

def value_nn(state, state_dim, initializer):
    """
    state_dim -relu-> 64 -> 1
    state -> value_estimated
    """
    w1 = tf.get_variable('w1', [state_dim, 64], initializer = initializer)
    b1 = tf.get_variable('b1', [64], initializer = tf.constant_initializer(0.0))
    h1 = tf.nn.relu(tf.matmul(state,w1) + b1)
    w2 = tf.get_variable('w2', [64,1], initializer = initializer)
    b2 = tf.get_variable('b2', [1], initializer = tf.constant_initializer(0.0))
    value_estimated = tf.matmul(h1, w2) + b2
    return tf.squeeze(value_estimated)

def q_network(state, state_dim, action_space, initializer):
    """
    state_dim -relu-> 128 -relu-> 64 -> action_space
    state -> [w1,b1,w2,b2,w3,b3,action_values]
    """
    w1 = tf.get_variable('w1', [state_dim, 128], initializer=initializer)
    b1 = tf.get_variable('b1', [128], initializer = tf.constant_initializer(0))
    h1 = tf.nn.relu(tf.matmul(state, w1) + b1)
    w2 = tf.get_variable('w2', [128, 64], initializer = initializer)
    b2 = tf.get_variable('b2', [64], initializer = tf.constant_initializer(0))
    h2 = tf.nn.relu(tf.matmul(h1, w2) + b2)
    w3 = tf.get_variable('w3', [64, action_space], initializer = initializer)
    b3 = tf.get_variable('b3', [action_space], initializer = tf.constant_initializer(0))
    action_values = tf.matmul(h2, w3) + b3
    return [w1,b1,w2,b2,w3,b3,action_values]

In [58]:
train_pairs = [line for line in codecs.open(task_path+'train_pos','r',encoding='utf-8')]
test_pairs = train_pairs  #= [line for line in codecs.open(task_path+'sort_test.pairs','r',encoding='utf-8')]

#### sl_policy.py 

In [61]:
class SupervisedPolicy(object):
    def __init__(self, learning_rate = 0.001):
        self.initializer = tf.contrib.layers.xavier_initializer()
        with tf.variable_scope('supervised_policy'):
            self.state = tf.placeholder(tf.float32, [None, state_dim], name = 'state')
            self.action = tf.placeholder(tf.int32, [None], name = 'action')
            self.action_prob = policy_nn(self.state, state_dim, action_space, self.initializer)
            action_mask = tf.cast(tf.one_hot(self.action, depth = action_space), tf.bool)
            self.picked_action_prob = tf.boolean_mask(self.action_prob, action_mask)

            self.loss = tf.reduce_sum(-tf.log(self.picked_action_prob)) + \
                                    sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES, scope = 'supervised_policy'))
            self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
            self.train_op = self.optimizer.minimize(self.loss)

    def predict(self, state, sess = None):
        sess = sess or tf.get_default_session()
        return sess.run(self.action_prob, {self.state: state})

    def update(self, state, action, sess = None):
        sess = sess or tf.get_default_session()
        _, loss = sess.run([self.train_op, self.loss], {self.state: state, self.action: action})
        return loss

def sl_train(episodes=500):
    tf.reset_default_graph()
    policy_nn = SupervisedPolicy()
    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for episode in range(len(train_pairs) if len(train_pairs)<episodes else episodes):
            #import pdb;pdb.set_trace()
            print("Episode %d" % episode);print('Training Sample:', train_pairs[episode%episodes][:-1])
            sample = train_pairs[episode%episodes].split()
            try:
                good_episodes = teacher(sample[0], sample[1], 5, env, task_path+'graph.txt') # good_episodes from teacher
            except Exception as e:
                print('Cannot find a path');continue

            for item in good_episodes: # one episode one supervised batch*<state,action> to update theta
                state_batch,action_batch = [],[]
                for t, transition in enumerate(item):
                    state_batch.append(transition.state)
                    action_batch.append(transition.action)
                state_batch = np.squeeze(state_batch)
                state_batch = np.reshape(state_batch, [-1, state_dim])
                policy_nn.update(state_batch, action_batch)
        saver.save(sess, 'models/policy_supervised_' + relation)
        print('Model saved')


def sl_test(episodes=300):
    tf.reset_default_graph()
    policy_nn = SupervisedPolicy()
    print('len(test_pairs):',len(test_pairs),'test_episodes:',episodes)
    success = 0
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, 'models/policy_supervised_'+ relation);print('Model reloaded')
        for episode in range(episodes):
            print('Test sample %d: %s' % (episode,test_pairs[episode][:-1]))
            sample = test_pairs[episode].split()
            state_idx = [env.entity2id_[sample[0]], env.entity2id_[sample[1]], 0]
            for t in count():
                state_vec = env.idx_state(state_idx)
                action_probs = policy_nn.predict(state_vec)
                action_chosen = np.random.choice(np.arange(action_space), p = np.squeeze(action_probs))
                reward, new_state, done = env.interact(state_idx, action_chosen)
                if done or t == max_steps_test:
                    if done:
                        print('Success')
                        success += 1
                    print('Episode ends\n')
                    break
                state_idx = new_state

    print('Success persentage:', success/episodes)

In [ ]:
sl_train(5)

In [ ]:
sl_test(5)

#### policy_agent.py

In [80]:
class PolicyNetwork(object):
    def __init__(self, scope = 'policy_network', learning_rate = 0.001):
        self.initializer = tf.contrib.layers.xavier_initializer()
        with tf.variable_scope(scope):
            self.state = tf.placeholder(tf.float32, [None, state_dim], name = 'state')
            self.action = tf.placeholder(tf.int32, [None], name = 'action')
            # +target
            self.target = tf.placeholder(tf.float32, name = 'target')
            self.action_prob = policy_nn(self.state, state_dim, action_space, self.initializer)

            action_mask = tf.cast(tf.one_hot(self.action, depth = action_space), tf.bool)
            self.picked_action_prob = tf.boolean_mask(self.action_prob, action_mask)
            # +target
            self.loss = tf.reduce_sum(-tf.log(self.picked_action_prob)*self.target) + \
                                    sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES, scope=scope))
            self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
            self.train_op = self.optimizer.minimize(self.loss)

    def predict(self, state, sess = None):
        sess = sess or tf.get_default_session()
        return sess.run(self.action_prob, {self.state:state})

    def update(self, state, target, action, sess=None):
        sess = sess or tf.get_default_session()
        # +target
        feed_dict = { self.state: state, self.target: target, self.action: action  }
        _, loss = sess.run([self.train_op, self.loss], feed_dict)
        return loss


def REINFORCE(train_pairs, policy_nn, num_episodes):
    success = 0
    path_found = []
    for i_episode in range(num_episodes):
        start = time.time()
        print('Episode %d' % i_episode);print('Training sample: ', train_pairs[i_episode][:-1])
        sample = train_pairs[i_episode].split()
        state_idx = [env.entity2id_[sample[0]], env.entity2id_[sample[1]], 0]
        episode,state_batch_negative,action_batch_negative = [],[],[]
        for t in count():
            state_vec = env.idx_state(state_idx)
            action_probs = policy_nn.predict(state_vec)
            action_chosen = np.random.choice(np.arange(action_space), p = np.squeeze(action_probs))
            reward, new_state_idx, done = env.interact(state_idx, action_chosen)
            # the action fails for this step
            if reward == -1: 
                state_batch_negative.append(state_vec)
                action_batch_negative.append(action_chosen)
            new_state_vec = env.idx_state(new_state_idx)
            episode.append(Transition(state = state_vec,\
                                      action = action_chosen,\
                                      next_state = new_state_vec,\
                                      reward = reward))
            if done or t == max_steps:break
            state_idx = new_state_idx
        # Discourage the agent when it choose an invalid step
        if len(state_batch_negative) != 0:
            print('Penalty to invalid steps:',len(state_batch_negative))
            policy_nn.update(np.reshape(state_batch_negative, (-1, state_dim)), -0.05, action_batch_negative)

        # If the agent success, do one optimization
        def update_episode(policy_nn,episode,total_reward):      
            state_batch = []
            action_batch = []
            for t, transition in enumerate(episode):
                if transition.reward == 0:
                    state_batch.append(transition.state)
                    action_batch.append(transition.action)
            policy_nn.update(np.reshape(state_batch,(-1,state_dim)), total_reward, action_batch)
            
        if done == 1:
            print('Success')
            path_found.append(path_clean(' -> '.join(env.path)))
            success += 1
            length_reward,global_reward = 1/len(env.path),1
            total_reward = 0.1*global_reward + 0.9*length_reward
            update_episode(policy_nn,episode,total_reward)
        else:
            global_reward = -0.05
            update_episode(policy_nn,episode,global_reward)
            print('Failed, Do one teacher guideline')
            try:
                good_episodes = teacher(sample[0], sample[1], 1, env, task_path+'graph.txt')
                [update_episode(policy_nn,episode,1) for episode in good_episodes]
            except Exception as e:
                print('Teacher guideline failed')
        print('Episode time: ',time.time() - start)
    print('Success percentage:',success/num_episodes)
    # reset env path
    env.path,env.path_relations = [],[]
    # store path stats
    path_found_relation = [' -> '.join([rel for ix,rel in enumerate(path.split(' -> ')) if ix%2 == 0]) \
                                                                                 for path in path_found]
    relation_path_stats = sorted(Counter(path_found_relation).items(),key = lambda x:x[1],reverse=True)
    with codecs.open('./tasks/'+relation+'/path_stats.txt','w',encoding='utf-8') as f:
        [f.write(item[0]+'\t'+str(item[1])+'\n') for item in relation_path_stats]
        print('Path stats saved')

def rl_retrain(episodes=300):
    print('Start retraining');tf.reset_default_graph()
    policy_network = PolicyNetwork(scope = 'supervised_policy') # restore form parameters of supervised_policy
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, 'models/policy_supervised_' + relation);print("sl_policy restored")
        REINFORCE(train_pairs, policy_network, len(train_pairs) if len(train_pairs)<episodes else episodes)
        saver.save(sess, 'models/policy_retrained_' + relation)
    print('Retrained model saved')

def rl_test(episodes=500):
    tf.reset_default_graph()
    policy_network = PolicyNetwork(scope = 'supervised_policy') # restore form parameters of supervised_policy
    success = 0
    saver = tf.train.Saver()
    path_found = []
    path_set = set()

    with tf.Session() as sess:
        saver.restore(sess, 'models/policy_retrained_' + relation);print('Model reloaded')
        for episode in range(len(test_pairs) if len(test_pairs)<episodes else episodes):
            print('Test sample %d: %s' % (episode,test_pairs[episode][:-1]))
            sample = test_pairs[episode].split()
            state_idx = [env.entity2id_[sample[0]], env.entity2id_[sample[1]], 0]
            transitions = []
            for t in count():
                state_vec = env.idx_state(state_idx)
                action_probs = np.squeeze(policy_network.predict(state_vec))
                action_chosen = np.random.choice(np.arange(action_space), p = action_probs)
                reward, new_state, done = env.interact(state_idx, action_chosen)
                new_state_vec = env.idx_state(new_state)
                transitions.append(Transition(state = state_vec, action = action_chosen, next_state = new_state_vec, reward = reward))
                if done or t == max_steps_test:
                    if done:
                        success += 1;print("Success")
                        path_found.append(path_clean(' -> '.join(env.path)))
                    else:
                        print('Episode ends due to step limit')
                    break
                state_idx = new_state
            if done:
                if len(path_set) != 0:
                    path_found_embedding = [env.path_embedding(path.split(' -> ')) for path in path_set]
                    curr_path_embedding = env.path_embedding(env.path_relations)
                    path_found_embedding = np.reshape(path_found_embedding, (-1,embedding_dim))
                    cos_sim = cosine_similarity(path_found_embedding, curr_path_embedding)
                    diverse_reward = -np.mean(cos_sim)
                    print('diverse_reward', diverse_reward)
                    #total_reward = 0.1*global_reward + 0.8*length_reward + 0.1*diverse_reward 
                    state_batch = []
                    action_batch = []
                    for t, transition in enumerate(transitions):
                        if transition.reward == 0:
                            state_batch.append(transition.state)
                            action_batch.append(transition.action)
                    policy_network.update(np.reshape(state_batch,(-1,state_dim)), 0.1*diverse_reward, action_batch)
                path_set.add(' -> '.join(env.path_relations))
    print('Success persentage:', success/episodes)
    # env path reset
    env.path,env.path_relations = [],[]
    # store path to use 
    path_found_relation = [' -> '.join([rel for ix,rel in enumerate(path.split(' -> ')) if ix%2 == 0]) \
                                                                                 for path in path_found]
    relation_path_stats = sorted(Counter(path_found_relation).items(), key = lambda x:x[1], reverse=True)
    ranking_path = sorted([(path_stat[0],len(path_stat[0].split(' -> '))) \
                           for path_stat in relation_path_stats],\
                          key = lambda x:x[1])
    with codecs.open('./tasks/'+relation+'/path_to_use.txt','w',encoding='utf-8') as f:
        [f.write(item[0]+'\n') for item in ranking_path]
        print('path to use saved')

In [ ]:
rl_retrain(5)

In [ ]:
rl_test(5)

# Evaluate

#### fact_prediction_eval.py
```
def bfs_two
def get_features
ap:
TransE
TransR
RL
TransH
TransD
```

In [15]:
# dict
entity2id = dict([(line.split()[0],int(line.split()[1])) for line in \
                  codecs.open(task_path+'entity2id.txt','r',encoding='utf-8') if len(line.split()) == 2])
relation2id = dict([(line.split()[0],int(line.split()[1])) for line in \
                    codecs.open(task_path+'relation2id.txt','r',encoding='utf-8') if len(line.split()) == 2])

In [31]:
len(entity2id)

75492

In [32]:
len(relation2id)

400

In [129]:
# rel
rel = relation.replace("_", ":") # concept_athletehomestadium -> concept:athletehomestadium

In [22]:
# TransE
ent_vec_E = np.loadtxt(task_path+'entity2vec.unif')
rel_vec_E = np.loadtxt(task_path+'relation2vec.unif')
relation_vec_E = rel_vec_E[relation2id[rel],:] # TransE relation vector

In [34]:
relation2id[rel]

141

In [27]:
ent_vec_E.shape

(75492, 50)

In [30]:
rel_vec_E.shape

(400, 50)

In [28]:
relation_vec_E.shape

(50,)

In [131]:
# TransR
ent_vec_R = np.loadtxt(task_path+'entity2vec.bern')
rel_vec_R = np.loadtxt(task_path+'relation2vec.bern')
M_R = np.loadtxt(task_path+'A.bern') #投影矩阵
M_R = M_R.reshape([-1,50,50])
relation_vec_R = rel_vec_R[relation2id[rel],:]#关系向量TransR
M_R_vec = M_R[relation2id[rel],:,:]

In [125]:
np.loadtxt(task_path+'A.bern').shape

(20000, 50)

In [126]:
M_R.shape

(400, 50, 50)

In [127]:
M_R_vec.shape

(50, 50)

In [130]:
# TransH
ent_vec_H = np.loadtxt(task_path+'entity2vec.vec')
rel_vec_H = np.loadtxt(task_path+'relation2vec.vec')
M_H = np.loadtxt(task_path+'A.vec')
M_H = M_H.reshape([rel_vec_H.shape[0],-1])
relation_vec_H = d_r = np.expand_dims(rel_vec_H[relation2id[rel],:],1)
w_r = np.expand_dims(M_H[relation2id[rel],:],1)

In [128]:
M_H.shape

(400, 50)

In [138]:
# TransD
ent_vec_D = np.loadtxt(task_path+'entity2vec.vec_D')
rel_vec_D = np.loadtxt(task_path+'relation2vec.vec_D')
M_D = np.loadtxt(task_path+'A.vec_D')
ent_num = ent_vec_D.shape[0]
rel_num = rel_vec_D.shape[0]
rel_tran = M_D[0:rel_num,:]
ent_tran = M_D[rel_num:,:]
dim_D = ent_vec_D.shape[1]

r = np.expand_dims(rel_vec_D[relation2id[rel],:], 1)
r_p = np.expand_dims(rel_tran[relation2id[rel],:], 1)

In [50]:
def get_features(task_path,relation2id):
    stats = dict([(line.split('\t')[0],int(line.split('\t')[1])) for line in \
                  codecs.open(task_path+'path_stats.txt','r',encoding='utf-8') if len(line.split()) == 2])
    max_freq = max(stats.values()) #路径最大次数
    useful_paths = [] # ids path list
    named_paths = [] # rel_names path list
    paths = [line.rstrip() for line in codecs.open(task_path+'path_to_use.txt','r',encoding='utf-8')]
    for path in paths:
        # filter: not in stats and count less
        if path not in stats:
            continue
        elif max_freq > 1 and stats[path] < 2: 
            continue
        # filter: len(path)<=0
        if len(path.split(' -> ')) <= 10:
            pathIndex = []
            pathName = []
            relations = path.split(' -> ')
            for rel in relations:
                pathName.append(rel)
                rel_id = relation2id[rel]
                pathIndex.append(rel_id)
            useful_paths.append(pathIndex)
            named_paths.append(pathName)
    return useful_paths, named_paths

In [51]:
# features
id_path,name_path = get_features(task_path,relation2id)
path_weights = np.array([1.0/len(path) for path in name_path])

In [54]:
id_path

[[130], [254], [15], [193], [252], [35]]

In [53]:
name_path

[['concept:personleadsorganization'],
 ['concept:organizationhiredperson_inv'],
 ['concept:agentcollaborateswithagent_inv'],
 ['concept:journalistwritesforpublication'],
 ['concept:mutualproxyfor_inv'],
 ['concept:organizationterminatedperson_inv']]

In [55]:
path_weights

array([1., 1., 1., 1., 1., 1.])

In [62]:
# kb
kb = KB()       #知识库
kb_inv = KB()   #逆向
for line in codecs.open(task_path+'graph.txt'):
    e1,rel,e2 = line.split()
    kb.addRelation(e1,rel,e2)     # entities{entity1：Path{.relation.connected_entity}(relation, entity2)}
    kb_inv.addRelation(e2,rel,e1)

In [66]:
len(kb.entities)

75227

In [67]:
len(kb_inv.entities)

75227

In [65]:
kb.entities['concept_politicsblog_perspective']

[	concept:proxyfor	concept_book_new,
 	concept:locationlocatedwithinlocation_inv	concept_lake_new,
 	concept:atlocation_inv	concept_beverage_new]

In [70]:
kb_inv.entities['concept_politicsblog_perspective']

[	concept:proxyfor_inv	concept_book_new,
 	concept:locationlocatedwithinlocation	concept_lake_new,
 	concept:atlocation	concept_beverage_new]

In [146]:
# read_samples
# thing$concept_politician_sam_sullivan,thing$concept_city_whistler:-
def read_samples(task_path,pairs_file='train.pairs'):
    pairs,labels = [],[]
    for line in codecs.open(task_path+pairs_file,'r',encoding='utf-8'):
        e1 = line.split(',')[0].replace('thing$','')
        e2 = line.split(',')[1].split(':')[0].replace('thing$','')
        if (e1 not in kb.entities) or (e2 not in kb.entities):
            continue
        pairs.append((e1,e2))
        labels.append(1 if line[-2] == '+' else 0)
    return pairs,labels

In [147]:
test_pairs,test_labels = read_samples(task_path,'sort_test.pairs')

In [72]:
test_pairs[:5]

[('concept_actor_peter_king', 'concept_blog_sports_illustrated'),
 ('concept_celebrity_sam_phillips', 'concept_company_post'),
 ('concept_celebrity_sam_phillips', 'concept_company_sun'),
 ('concept_celebrity_sam_phillips', 'concept_company_sun_microsystems001'),
 ('concept_celebrity_sam_phillips', 'concept_newspaper_journal')]

In [74]:
len(test_pairs)

2698

In [73]:
test_labels[:5]

[1, 0, 1, 0, 0]

In [107]:
set().add('a')

In [120]:
def bi_path_search(e1,e2,path,kb,kb_inv):
    '''
    :the bidirectional search for reasoning
    e1,r1,ex,r2,e2
    path:start-><-end:r1,r2
    kb:->:e1,r1,ex|ex,r2,e2
    kb_inv:<-:ex,r1,e1|e2,r2,ex
    '''
    start,end = 0,len(path)
    left,right = set(),set()
    left.add(e1)
    right.add(e2)
    left_path,right_path = [],[]
    while(start < end):
        left_step = path[start]
        left_next = set()
        right_step = path[end-1]
        right_next = set()

        if len(left) < len(right):
            left_path.append(left_step)
            start += 1
            for entity in left: # BFS loop
                try:
                    for path_ in kb.getPathsFrom(entity):
                        if path_.relation == left_step:
                            left_next.add(path_.connected_entity)
                except Exception as e:
                    #print('len(left):',len(left),left,'not such entity')
                    return False
            left = left_next

        else: # start->end;kb->kb_inv
            right_path.append(right_step)
            end -= 1
            for entity in right:
                try:
                    for path_ in kb_inv.getPathsFrom(entity):
                        if path_.relation == right_step:
                            right_next.add(path_.connected_entity)
                except Exception as e:
                    #print('len(right):',len(right),right,'not such entity')
                    return False
            right = right_next
    #if len(right & left) != 0:
     #   print(left,right)
    return True if len(right & left) != 0 else False

In [178]:
# aps
def entity2vec(ent_vec,entity,M_vec=None,w_r=None):
    if isinstance(w_r,np.ndarray):
        ent = np.expand_dims(ent_vec[entity2id[entity],:],1)
        ent_ = ent - np.matmul(w_r.transpose(), ent)*w_r
        return ent_
    if isinstance(M_vec,np.ndarray):
        return np.matmul(ent_vec[entity2id[entity],:], M_vec)
    return ent_vec[entity2id[entity],:]

def l2_score(h,r,t):
    return -np.sum(np.square(h + r - t))

scores_E = [l2_score(entity2vec(ent_vec_E,pair[0]),relation_vec_E,entity2vec(ent_vec_E,pair[1])) for pair in test_pairs]
scores_R = [l2_score(entity2vec(ent_vec_R,pair[0],M_vec=M_R_vec),relation_vec_R,entity2vec(ent_vec_R,pair[1],M_vec=M_R_vec)) for pair in test_pairs]
scores_rl = [sum(path_weights*[int(bi_path_search(pair[0], pair[1], path, kb, kb_inv)) for path in name_path]) for pair in test_pairs]
scores_H = [l2_score(entity2vec(ent_vec_H,pair[0],w_r=w_r),relation_vec_H,entity2vec(ent_vec_H,pair[1],w_r=w_r)) for pair in test_pairs]

def score_D(ent_vec_D,ent_tran,pair):
    h = np.expand_dims(ent_vec_D[entity2id[pair[0]],:], 1)
    h_p = np.expand_dims(ent_tran[entity2id[pair[0]],:], 1)
    t = np.expand_dims(ent_vec_D[entity2id[pair[1]],:], 1)
    t_p = np.expand_dims(ent_tran[entity2id[pair[1]],:], 1)
    M_rh = np.matmul(r_p, h_p.transpose()) + np.identity(dim)
    M_rt = np.matmul(r_p, t_p.transpose()) + np.identity(dim)
    score = - np.sum(np.square(M_rh.dot(h) + r - M_rt.dot(t)))
    return score
    
scores_D =[score_D(ent_vec_D,ent_tran,pair) for pair in test_pairs] 

def ap(scores,test_labels):
    # evaluate rank quality
    # 1 1 1 0 0 0 better than 0 0 0 1 1 1
    rank_stats = sorted(zip(scores, test_labels),key = lambda x:x[0], reverse=True)
    correct = 0
    ranks = []
    for idx, item in enumerate(rank_stats):
        if item[1] == 1:
            correct += 1
            ranks.append(correct/(1.0+idx)) # append precision
    return np.mean(ranks) if len(ranks) != 0 else 0

models = {
    'E':ap(scores_E, test_labels),
    'R':ap(scores_R, test_labels),
    'rl':ap(scores_rl, test_labels),
    'H':ap(scores_H, test_labels),
    'D':ap(scores_D, test_labels)
}

In [179]:
models

{'D': 0.23670935347580135,
 'E': 0.23067132455999617,
 'H': 0.2476364956250818,
 'R': 0.28690542314091916,
 'rl': 0.48808056673088396}

#### link_prediction_eval.sh

#### evaluate.py

In [150]:
from sklearn import linear_model
from keras.models import Sequential 
from keras.layers import Dense, Activation

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [151]:
def train(kb, kb_inv, name_path):
    train_pairs,train_labels = read_samples(task_path,'train.pairs')
    training_features = [[int(bi_path_search(pair[0], pair[1], path, kb, kb_inv)) for path in name_path] for pair in train_pairs]
    model = Sequential()
    input_dim = len(name_path)
    model.add(Dense(1, activation='sigmoid' ,input_dim=input_dim))
    model.compile(optimizer = 'rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(training_features, train_labels, nb_epoch=300, batch_size=128)
    return model

In [191]:
y_scores = [model.predict(np.reshape([int(bi_path_search(pair[0], pair[1], path, kb, kb_inv)) for path in name_path],[1,-1])) for pair in test_pairs]
ap_link = ap(y_scores,test_labels)
mAP = np.mean(ap_link)
print('mAP:',mAP)

mAP: 0.5511950207386302
